In [3]:
import neuraltda.simpComp as sc
import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import pickle
reload(sa)

<module 'neuraltda.spectralAnalysis' from '/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/spectralAnalysis.pyc'>

In [2]:
blockPath = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/'
bdf = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/binned_data/20170128T005306Z/20170128T005306Z-10.0-5.0.binned'
scgf = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/scg/20170128T005306Z-10.0-5.0.scg'

In [ ]:
thresh = 15.0
sa.computeChainGroups(blockPath, bdf, thresh)

In [6]:
with open(scgf, 'r') as scgff:
    scg = pickle.load(scgff)

In [9]:
scg['E_scaled_burung'][11]

[[()],
 [(0,),
  (3,),
  (4,),
  (5,),
  (6,),
  (7,),
  (8,),
  (9,),
  (10,),
  (11,),
  (12,),
  (13,),
  (14,),
  (15,),
  (16,),
  (19,),
  (20,),
  (21,),
  (22,),
  (23,),
  (24,),
  (27,),
  (28,),
  (29,),
  (30,),
  (31,),
  (32,),
  (33,),
  (34,),
  (36,),
  (38,),
  (40,),
  (41,),
  (42,),
  (43,),
  (44,),
  (45,),
  (46,)],
 [(0, 4),
  (0, 10),
  (0, 11),
  (0, 12),
  (0, 13),
  (0, 16),
  (0, 21),
  (0, 30),
  (0, 32),
  (0, 42),
  (0, 45),
  (0, 46),
  (3, 4),
  (3, 13),
  (3, 21),
  (4, 7),
  (4, 12),
  (4, 13),
  (4, 21),
  (4, 24),
  (4, 30),
  (4, 31),
  (4, 32),
  (4, 42),
  (5, 9),
  (5, 21),
  (5, 24),
  (5, 30),
  (5, 42),
  (5, 46),
  (6, 13),
  (6, 21),
  (6, 23),
  (6, 28),
  (6, 30),
  (6, 33),
  (6, 34),
  (6, 36),
  (6, 42),
  (6, 43),
  (7, 8),
  (7, 9),
  (7, 10),
  (7, 13),
  (7, 21),
  (7, 32),
  (7, 41),
  (7, 42),
  (7, 46),
  (8, 9),
  (8, 13),
  (8, 15),
  (8, 19),
  (8, 21),
  (8, 24),
  (8, 32),
  (8, 36),
  (8, 42),
  (9, 10),
  (9, 13),
  (9,